In [1]:
import os
import pandas as pd
import psycopg2
from psycopg2 import OperationalError
from datetime import datetime, timedelta

In [ ]:
# Function to connect to the PostgreSQL database
def connect_to_db():
    try:
        conn = psycopg2.connect(
            dbname="timeseries",
            user="postgres",
            password="diploma",
            host="localhost",
            port="5432"
        )
        print("Connected to the database.")
        return conn
    except OperationalError as e:
        print(f"Error: {e}")
        return None

In [3]:
# Function to create the Timeseries table and fill it with data
def create_and_fill_timeseries_table(conn, csv_dir):
    try:
        cursor = conn.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Timeseries (
                id VARCHAR PRIMARY KEY,
                original_timeseries_id VARCHAR,
                type VARCHAR(50),
                measure_id INT,
                source VARCHAR(100),
                additional_properties JSONB
            );
        """)
        print("Timeseries table created successfully.")

        # Iterate through CSV files in the directory
        for file in os.listdir(csv_dir):
            if file.endswith(".csv"):
                timeseries_id = os.path.splitext(os.path.basename(file))[0]
                cursor.execute("""
                    INSERT INTO Timeseries (id)
                    VALUES (%s)
                    ON CONFLICT (id) DO NOTHING;
                """, (timeseries_id,))
        conn.commit()
        print("Data inserted into Timeseries table successfully.")

    except Exception as e:
        print(f"Error: {e}")

In [4]:
# Function to create the Signal_Values_timestamp table and insert data into it
def create_and_insert_signal_values_table(conn, csv_dir):
    try:
        cursor = conn.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Signal_Values_timestamp (
                timeseries_id VARCHAR,
                timestamp TIMESTAMPTZ,
                value FLOAT,
                additional_properties VARCHAR,
                FOREIGN KEY (timeseries_id) REFERENCES Timeseries(id)
            );
        """)
        print("Signal_Values_timestamp table created successfully.")

        # Iterate through CSV files in the directory
        for file in os.listdir(csv_dir):
            if file.endswith(".csv"):
                csv_file = os.path.join(csv_dir, file)
                timeseries_id = os.path.splitext(os.path.basename(csv_file))[0]
                
                # Read CSV file into pandas DataFrame
                df = pd.read_csv(csv_file)

                # Define the starting time
                start_time = datetime(2024, 11, 10, 3, 0, 0, 0) + timedelta(hours=1)

                # Insert DataFrame rows into the table
                for _, row in df.iterrows():
                    # Use the first column as milliseconds
                    milliseconds = int(row.iloc[0])
                    timestamp = start_time + timedelta(milliseconds=milliseconds)
                    additional_properties = str(row['EmotionRange'])
                    
                    cursor.execute("""
                        INSERT INTO Signal_Values_timestamp (timeseries_id, timestamp, value, additional_properties)
                        VALUES (%s, %s, %s, %s)
                    """, (timeseries_id, timestamp, row['Value'], additional_properties))
        conn.commit()
        print("Data inserted into Signal_Values_timestamp table successfully.")

    except Exception as e:
        print(f"Error: {e}")


In [5]:
# Main function
def main():
    # Connect to the PostgreSQL database
    conn = connect_to_db()
    if conn is None:
        return

    # Directory containing CSV files
    csv_dir = "prepared_data"

    # Create and fill the Timeseries table
    create_and_fill_timeseries_table(conn, csv_dir)

    # Create and insert data into the Signal_Values_timestamp table
    create_and_insert_signal_values_table(conn, csv_dir)

    # Close the database connection
    conn.close()
    print("Database connection closed.")

if __name__ == "__main__":
    main()

Connected to the database.
Timeseries table created successfully.
Data inserted into Timeseries table successfully.
Signal_Values_timestamp table created successfully.
Data inserted into Signal_Values_timestamp table successfully.
Database connection closed.
